# Dependencies

## Installation

In [ ]:
%pip install pip --upgrade -q
%pip install pandas -q
%pip install numpy -q
%pip install matplotlib -q
%pip install folium -q
%pip install geopy -q
%pip install geopandas -q

## Importing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd

# Loading Data

In [ ]:
DTYPE = {
    "IBGE-code": str, 
    "IBGE-state-code": str, 
    "IBGE-municipality-code": str,
}

GEO_DATA_PATH = "./data/generated/geo/municipalities_geodata.geojson"
SUS_DATA = "./data/generated/sus/mental_diseases_jan2024-dec2024.csv"

In [ ]:
from numpy import dtype

geo_data = gpd.read_file(GEO_DATA_PATH, encoding="utf-8")
sus_data = pd.read_csv(SUS_DATA, encoding="utf-8", sep=";", dtype=DTYPE)

In [ ]:
# GeoJson dont accept dtype "object" in columns with numbers, so we need to convert them to str
geo_data["IBGE-code"] = geo_data["IBGE-code"].astype(str)
geo_data["IBGE-state-code"] = geo_data["IBGE-state-code"].astype(str)
geo_data["IBGE-municipality-code"] = geo_data["IBGE-municipality-code"].astype(str)

In [ ]:
# Display the first few rows of the GeoDataFrame
geo_data.head(5)

In [ ]:
# Display the first few rows of the Sus_data DataFrame
sus_data.head(5)

# Joining Data

In [ ]:
# Joining the SUS data with the geographical data to get the municipalities with reports
municipalities_with_reports = geo_data.merge(sus_data, how="inner", on=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code"])
municipalities_with_reports = gpd.GeoDataFrame(
    municipalities_with_reports[["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-name", "municipality-state", "internacoes"]],
    geometry=municipalities_with_reports["geometry"],
)
municipalities_with_reports.head(5)

In [ ]:
# Joining the SUS data with the geographical data to get the municipalities without reports
municipalities_without_reports = geo_data.copy()

municipalities_without_reports = municipalities_without_reports.merge(sus_data, how="left", on=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code"])
municipalities_without_reports = municipalities_without_reports[municipalities_without_reports["internacoes"].isna()]
municipalities_without_reports = municipalities_without_reports.drop(columns=["internacoes"])
municipalities_without_reports.head(5)

In [ ]:
# Saving the dataframes
MUNICIPALITIES_WITH_REPORTS_FILE_PATH = "./data/generated/mixed/municipalities_with_reports_jan2024-dec2024.csv"
MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH = "./data/generated/mixed/municipalities_without_reports_jan2024-dec2024.csv"
municipalities_with_reports.to_csv(MUNICIPALITIES_WITH_REPORTS_FILE_PATH, sep=";", index=False, encoding="utf-8")
municipalities_without_reports.to_csv(MUNICIPALITIES_WITHOUT_REPORTS_FILE_PATH, sep=";", index=False, encoding="utf-8")



# Generating Map

In [ ]:
BRAZILIA_CORDINATES = [-15.7801, -47.9292]
m = folium.Map(location=BRAZILIA_CORDINATES, zoom_start=4, tiles='cartodb positron',)

In [ ]:
# All municipalities in Brazil
folium.GeoJson(
    geo_data,
    name="All Municipalities",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "white",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)


In [ ]:
# Municipalities without reports
folium.GeoJson(
    municipalities_without_reports,
    name="Municipalities without reports",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "red",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
).add_to(m)

In [ ]:
# Municipalities with reports
folium.GeoJson(
    municipalities_with_reports,
    name="Municipalities with reports",
    style_function=lambda x: {
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
        "fillColor": "green",
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"],
        aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name" , "Internacoes"],
        localize=True,
        sticky=False,
        labels=True,
        style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    ),
    

).add_to(m)

In [ ]:
folium.LayerControl().add_to(m)
m

In [ ]:
BRAZILIA_CORDINATES = [-15.7801, -47.9292]
m = folium.Map(location=BRAZILIA_CORDINATES, zoom_start=4, tiles='cartodb positron')

In [ ]:
# Adding a choropleth map to show the number of reports in each municipality



folium.Choropleth(
    name ="Municipalities with reports - Hospitalizations",
    geo_data=geo_data,
    data=municipalities_with_reports,
    columns=["IBGE-code", "internacoes"],
    key_on="feature.properties.IBGE-code",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Internações",
    # tooltip=folium.features.properties.GeoJsonTooltip(
    #     fields=["IBGE-code", "IBGE-state-code", "IBGE-municipality-code", "municipality-state", "municipality-name", "internacoes"],
    #     # aliases=["IBGE code", "IBGE state code", "IBGE municipality code", "Municipality state", "Municipality name" , "Internacoes"],
    #     # localize=True,
    #     # sticky=False,
    #     # labels=True,
    #     # style="background-color: white; color: black; font-family: Arial; font-size: 12px; padding: 5px;",
    # ),
    tooltip = folium.Tooltip("teste")
).add_to(m)

In [ ]:
m

# Testing

In [ ]:
# Display the number of municipalities with reports and without reports
print(f"Municipalities with reports: {len(municipalities_with_reports)}")
print(f"Municipalities without reports: {len(municipalities_without_reports)}")
# Display the total number of municipalities in Brazil
print(f"Total municipalities in Brazil: {len(geo_data)}")
print(f"Total municipalities with reports: {len(municipalities_with_reports)}")
print(f"Total municipalities without reports: {len(municipalities_without_reports)}")
